# torchvision 
——这是 PyTorch 专门为 计算机视觉 (CV) 提供的工具包。

它主要包括三大类功能：

- 数据集 (torchvision.datasets)

- 图像变换 (torchvision.transforms)

- 模型 (torchvision.models)

另外还有一些图像处理工具和可视化方法。

---
## 1. torchvision.datasets

提供了很多常用的` 图像数据集接口`，可以直接下载、加载并返回 PyTorch 的` Dataset `对象，和` DataLoader `配合使用。

常见的数据集：
| 数据集                    | 任务      | 描述               |
| ---------------------- | ------- | ---------------- |
| **MNIST**              | 图像分类    | 手写数字 (28×28 灰度图) |
| **CIFAR10 / CIFAR100** | 图像分类    | 小图片 (32×32，RGB)  |
| **FashionMNIST**       | 图像分类    | 类似 MNIST，但内容是衣物  |
| **ImageNet**           | 图像分类    | 大规模数据集 (1000 类)  |
| **COCO**               | 目标检测/分割 | 常用的检测 & 分割数据集    |
| **VOC**                | 目标检测/分割 | PASCAL VOC 数据集   |

示例：

In [ ]:
import torchvision
import torchvision.transforms as transforms

# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 下载 & 加载 MNIST
trainset = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform
)


## 2. torchvision.transforms

用于` 数据增强 `/` 预处理`，把` PIL.Image `或` Tensor `转换为适合神经网络的输入。

常见操作：
| 方法                                       | 作用                     |
| ---------------------------------------- | ---------------------- |
| `transforms.Resize(size)`                | 调整图片大小                 |
| `transforms.CenterCrop(size)`            | 裁剪中心区域                 |
| `transforms.RandomCrop(size)`            | 随机裁剪                   |
| `transforms.RandomHorizontalFlip(p=0.5)` | 随机水平翻转                 |
| `transforms.RandomRotation(degrees)`     | 随机旋转                   |
| `transforms.ColorJitter()`               | 随机改变亮度/对比度/饱和度         |
| `transforms.ToTensor()`                  | 转换为 Tensor，归一化到 \[0,1] |
| `transforms.Normalize(mean, std)`        | 标准化 `(x-mean)/std`     |
| `transforms.Compose([...])`              | 串联多个变换                 |

示例：

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # 统一大小
    transforms.RandomHorizontalFlip(),  # 随机翻转
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # 标准化
])


## 3. torchvision.models

提供了很多` 预训练模型`（在 ImageNet 上训练过），可以直接拿来` 迁移学习 `或` 特征提取`。

常见模型：

- 分类模型：

    - ResNet (18/34/50/101/152)
    
    - VGG (11/13/16/19)
    
    - DenseNet
    
    - AlexNet
    
    - MobileNet
    
    - EfficientNet
    
    - Vision Transformer (ViT)

- 检测 / 分割模型：

    - Faster R-CNN
    
    - Mask R-CNN
    
    - RetinaNet
    
    - SSD
    
    - DeepLabV3
    
    - FCN

使用预训练模型

In [ ]:
import torchvision.models as models

# 加载预训练 ResNet18
model = models.resnet18(pretrained=True)

# 修改最后一层（适应 10 分类任务）
import torch.nn as nn
model.fc = nn.Linear(model.fc.in_features, 10)


## 4. torchvision.io / utils

`torchvision.io`：图像/视频的读写

In [ ]:
from torchvision.io import read_image
img = read_image("example.jpg")  # 直接读取为 Tensor


`torchvision.utils`：可视化工具

In [ ]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

grid = make_grid(img_batch, nrow=8, normalize=True)
plt.imshow(grid.permute(1, 2, 0))  # C,H,W -> H,W,C
plt.show()


## 5. 总结

✅ torchvision 是 PyTorch 的视觉工具包，主要作用：

- datasets → 快速加载常见数据集 (MNIST, CIFAR, COCO, VOC, ImageNet)

- transforms → 数据增强 & 预处理 (Resize, Flip, Normalize...)

- models → 提供预训练模型 (ResNet, VGG, ViT, Faster-RCNN...)

- io / utils → 图像读写 & 可视化工具

# 🔥 迁移学习示例：ResNet18 + CIFAR-10

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np

# ===============================
# 1. 数据准备
# ===============================
transform_train = transforms.Compose([
    transforms.Resize(224),  # ResNet 输入要求 224x224
    transforms.RandomHorizontalFlip(),  # 随机水平翻转
    transforms.RandomCrop(224, padding=4),  # 随机裁剪
    transforms.ToTensor(),  # 转为 tensor 张量数据类型
    transforms.Normalize((0.5, 0.5, 0.5),  # 归一化
                         (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.Resize(224),   # 重新设置图片的尺寸
    transforms.ToTensor(),    
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train
)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test
)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

classes = trainset.classes  # CIFAR-10 类别

# ===============================
# 2. 定义模型（迁移学习）
# ===============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载预训练模型
model = models.resnet18(pretrained=True)

# 冻结前面的卷积层参数（只训练最后全连接层）
for param in model.parameters():
    param.requires_grad = False

# 修改最后一层：CIFAR-10 共有 10 类
model.fc = nn.Linear(model.fc.in_features, 10)

model = model.to(device)

# ===============================
# 3. 损失函数 & 优化器 & 学习率调度器
# ===============================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)  # 只优化最后一层
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # 每 10 epoch 学习率 ×0.1

# ===============================
# 4. 训练 & 验证
# ===============================
train_losses, val_losses = [], []
train_accs, val_accs = [], []

def train(epoch):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for inputs, targets in trainloader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()   # 清空梯度
        outputs = model(inputs) # 预测（前向传播）
        loss = criterion(outputs, targets)  # 计算损失
        loss.backward()         # 反向梯度传播
        optimizer.step()        # 根据梯度更新 权重

        running_loss += loss.item()    # 统计训练损失
        _, predicted = outputs.max(1)  # 提取预测类别的索引
        total += targets.size(0)       # 统计训练总样本
        correct += predicted.eq(targets).sum().item()  # 统计准确率
 
    epoch_loss = running_loss / len(trainloader)  # 计算每个epoch损失 
    epoch_acc = 100. * correct / total            # 计算每个epoch准确率
    return epoch_loss, epoch_acc


def validate(epoch):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    epoch_loss = running_loss / len(testloader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc


# ===============================
# 5. 主训练循环
# ===============================
num_epochs = 5
for epoch in range(num_epochs):
    train_loss, train_acc = train(epoch)
    val_loss, val_acc = validate(epoch)
    scheduler.step()   # 更新学习率

    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% "
          f"| Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

# ===============================
# 6. 可视化 Loss & Acc 曲线
# ===============================
epochs = range(1, num_epochs+1)

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(epochs, train_losses, label="Train Loss")
plt.plot(epochs, val_losses, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss Curve")
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs, train_accs, label="Train Acc")
plt.plot(epochs, val_accs, label="Val Acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("Accuracy Curve")
plt.legend()

plt.show()

# ===============================
# 7. 测试集可视化预测结果
# ===============================
def imshow(img):
    img = img / 2 + 0.5  # 去归一化
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# 随机获取一些测试样本
dataiter = iter(testloader)
images, labels = next(dataiter)

# 显示图像
imshow(torchvision.utils.make_grid(images[:8]))
print("GroundTruth: ", " ".join(f"{classes[labels[j]]}" for j in range(8)))

# 模型预测
images = images.to(device)
outputs = model(images)
_, predicted = outputs.max(1)

print("Predicted: ", " ".join(f"{classes[predicted[j]]}" for j in range(8)))


## 🚀 运行效果

训练日志：

- 每个 epoch 输出 Train Loss/Acc 和 Val Loss/Acc

- Loss & Accuracy 曲线：可视化对比训练 & 验证过程

- 测试样本可视化：展示真实标签 vs 模型预测